## 기사를 통한 kobert 모델 생성

In [ ]:
# !pip uninstall torch torchvision
# !pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 -f https://download.pytorch.org/whl/torch_stable.html
#pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
from kobert import get_pytorch_kobert_model
from tqdm import tqdm
import os
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW # 인공지능 모델의 초기값 지정 함수를 아담으로 지정한다.
from transformers.optimization import get_cosine_schedule_with_warmup

from sklearn.model_selection import train_test_split


max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 15
max_grad_norm = 1
log_interval = 100
learning_rate =  5e-5
device = torch.device("cuda:0")
bertmodel, vocab = get_pytorch_kobert_model()

ModuleNotFoundError: No module named 'torch'

In [ ]:

df = pd.read_csv('/Users/stillssi/Desktop/ASSJ/data/article_preprocessed.csv')

In [ ]:
#1과0 일정한 비율로 데이터 뽑아서 train, test data 각각 return
def same_sampling(df):
  one = df[df['label'] == 1].sample(n=40000)
  zero = df[df['label']== 0].sample(n=40000)
  
  new_df = pd.concat([one, zero], axis=0)
  new_df = new_df.sample(frac=1)
  
  new_df = new_df[['prep','label']]
  new_df['prep'].dropna(axis=0, inplace=True)
  
  data = new_df[['prep']]
  label = new_df[['label']]

  
  X_train, X_test, y_train, y_test = train_test_split(data, label,test_size=0.3,shuffle=True, random_state=1004)
  train_df = pd.concat([X_train, y_train], axis=1)
  test_df = pd.concat([X_test, y_test], axis=1)
  
  train_df['prep'] = train_df['prep'].apply(lambda x: str(x))
  test_df['prep'] = test_df['prep'].apply(lambda x: str(x))
  
  return train_df, test_df

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

def gpu_count():
    n_devices = torch.cuda.device_count()
    print(n_devices)

    for i in range(n_devices):
        print(torch.cuda.get_device_name(i))

def new_softmax(a) : 
    c = np.max(a) # 최댓값
    exp_a = np.exp(a-c) #                                                                                                                                                     각각의 원소에 최댓값을 뺀 값에 exp를 취한다. (이를 통해 overflow 방지)
    sum_exp_a = np.sum(exp_a)
    y = (exp_a / sum_exp_a) * 100
    return np.round(y, 3)

#==================================================
#모델링을 위한 데이터 셋
class BERTDataset(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                pad, pair):
      transform = nlp.data.BERTSentenceTransform(
          bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

      self.sentences = [transform([i[sent_idx]]) for i in dataset]
      self.labels = [np.int32(i[label_idx]) for i in dataset]

  def __getitem__(self, i):
      return (self.sentences[i] + (self.labels[i], ))

  def __len__(self):
      return (len(self.labels))  

#===================================================
#모델링 클래스
class BERTClassifier(nn.Module): ## 클래스를 상속
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [1]:
def modeling(df, train):
    '''
    모델링 함수
    '''
    #BERT 모델 불러오기
    model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
    #optimizer와 schedule 설정
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    
    train_df, test_df = same_sampling(df)
    data_train = BERTDataset(train_df.values, 0, 1, tok, max_len, True, False)
    data_test = BERTDataset(test_df.values, 0, 1, tok, max_len, True, False)
    train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
    test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

    optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
    loss_fn = nn.CrossEntropyLoss()

    t_total = len(train_dataloader) * num_epochs
    warmup_step = int(t_total * warmup_ratio)

    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


    for e in range(num_epochs):
        train_acc = 0.0
        test_acc = 0.0
        model.train()
        train_df, test_df = same_sampling(df)
        
        data_train = BERTDataset(train_df.values, 0, 1, tok, max_len, True, False)
        data_test = BERTDataset(test_df.values, 0, 1, tok, max_len, True, False)
        train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
        test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)
        
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
            optimizer.zero_grad()
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length= valid_length
            label = label.long().to(device)
            out = model(token_ids, valid_length, segment_ids)
            loss = loss_fn(out, label)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            scheduler.step()  # Update learning rate schedule
            train_acc += calc_accuracy(out, label)
            if batch_id % log_interval == 0:
                print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
        print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
        
        model.eval()
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length= valid_length
            label = label.long().to(device)
            out = model(token_ids, valid_length, segment_ids)
            test_acc += calc_accuracy(out, label)
        print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    return model, optimizer

In [3]:
def save_model(model, optimizer):
    '''
    모델 저장 함수
    '''
    PATH = '/content/drive/MyDrive/Model/KoBert/' # google 드라이브 연동 해야함. 관련코드는 뺐음
    torch.save(model, PATH + 'KoBERT_news.pt')  # 전체 모델 저장
    torch.save(model.state_dict(), PATH + 'model_state_dict.pt')  # 모델 객체의 state_dict 저장
    torch.save({
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }, PATH + 'all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능

In [ ]:
import pandas as pd

## **댓글** 감성 분석 실시

In [ ]:
naver = pd.read_csv('preprocessed_naver_news_comment.csv')
naver = naver[['Content']]
naver['label'] = 1


youtube = pd.read_csv('preprocessed_youtube.csv')
youtube = youtube[['comment']]
youtube['label'] = 1

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

In [ ]:
## 학습 모델 로드
def load_model():
    PATH = '/content/drive/MyDrive/Model/KoBert/'
    model = torch.load(PATH + 'KoBERT_news.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
    model.load_state_dict(torch.load(PATH + 'model_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장
    return model


def sentiment_analysis(df, model):
    data_test = BERTDataset(df.values, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)
    model.eval()
    
    id= 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)
        
        out = model(token_ids, valid_length, segment_ids)
        probability = []
        print(out.shape)

        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            min_v = min(logits)
            total = 0
            
            logits = np.round(new_softmax(logits), 3).tolist()
            for logit in logits:
                # print(logit)
                probability.append(np.round(logit, 3))

            if np.argmax(logits) == 0:  emotion = 0
            else: emotion = 1
            df['label'].iloc[id] = emotion
            
            id +=1

    return df

In [ ]:
## 학습 모델 저장
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

model, optimizer = modeling()
save_model(model, optimizer)

In [ ]:

# 감정 분석 수행
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
model = load_model()
naver_label =sentiment_analysis(naver, model)
youtube_label  = sentiment_analysis(youtube, model)


In [ ]:
naver = pd.concat([naver,naver_label['label']], axis=1)
youtube = pd.concat([youtube,youtube_label['label']], axis=1)

In [ ]:
naver.to_csv('naver_comment_labeled.csv')
youtube.to_csv('youtube_comment_labeled.csv')